In [6]:
import numpy as np
import pandas as pd

In [13]:
import pandas as pd

# Specify your file path
file_path = '../crawling/file_content_list.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df = df.dropna(subset=['Content'])

# Display the first few rows of the DataFrame
print(df.head(10))


                                               Title  Category        Date  \
0                           SKKU AI Colloquium  2023      [기타]  2023-09-14   
1                     실감미디어공학과 세미나 안내 (9/14, 10:30~)  [행사/세미나]  2023-09-12   
2                     [공개SW특강] 대규모 오픈소스 프로그램 쉽게 분석하기  [행사/세미나]  2023-09-12   
3                           2023 국방 AI 경진대회 (MAICON)      [기타]  2023-09-11   
4             지능형소프트웨어학과(대학원) 진학상담회 및 산학장학생 안내(9/18)  [행사/세미나]  2023-09-11   
5                               2023년-2학기 실험실습 TA 모집   [채용/모집]  2023-09-08   
6               디지털인재얼라이언스 가입 기업 대상 인턴십 희망 기업 안내 리스트   [채용/모집]  2023-09-07   
7                        2023년-2학기 1만라인 TA 모집(~9/12)   [채용/모집]  2023-09-06   
8                                   [SPARK 12기 참가모집]   [채용/모집]  2023-09-01   
9  [성균웹진] 미국 교환학생 후기 - The University of Texas at...      [기타]  2023-08-30   

                                             Content  \
0  SKKU AI Colloquium 2023Talking Sharing and Lea...   
1  실감미디어공학과 세미나가 아래와 같이 개최되오니

In [8]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import faiss

In [9]:
# 2. Vectorize Korean text columns using KoBERT
class KoBertEmbedding:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
        self.model = BertModel.from_pretrained('monologg/kobert')
        self.model.eval()

    def get_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs['last_hidden_state'][:,0,:].numpy()


In [12]:
print(df["Content"])

실감미디어공학과 세미나가 아래와 같이 개최되오니 많은 참여 부탁드립니다일시  9월 14일 목 1330장소  국제관 9B312호강연제목  Understanding Human in the 3D Space강연자  문경식 MetaPostdoctoral Research ScientistZOOM  ID 591 389 1581  PW 908353


In [14]:
# Vectorize using KoBERT
kobert = KoBertEmbedding()

df["Vectors"] = df["Content"].apply(kobert.get_embedding)
vectors = np.vstack(df["Vectors"].values)

# 3. Create a Faiss index and add the vectors
d = vectors.shape[1]
index = faiss.IndexFlatL2(d)
index.add(vectors)

In [23]:
def search_k_nearest(text, K=10):
    vector = kobert.get_embedding(text)
    distances, indices = index.search(vector, K)
    for i in range(K):
        print(f"Rank {i+1}: {df.iloc[indices[0][i]]['Title']} (Distance: {distances[0][i]:.4f})")


In [24]:
search_k_nearest("학사 일정")

Rank 1: [펄어비스] 2022 펄어비스 채용연계형 겨울인턴십 (10.15(토)~11.06(일)) (Distance: 0.0000)
Rank 2: 2022년 산학협력프로젝트 참여 신청 공고~4/7(목)까지 (Distance: 0.0000)
Rank 3: 현대자동차 SW/데이터 엔지니어 신입 채용(~5/1) (Distance: 13.6295)
Rank 4: ㈜로보티즈 웹 개발 외 채용 (Distance: 13.6295)
Rank 5: [대한상공회의소] 2022년 신입직원 채용 (~10/1(금)) (Distance: 13.6295)
Rank 6: [그래피카] 그래피카 채용 공고 (Distance: 13.6295)
Rank 7: [한화큐셀] Q-Global Talent 전형 대상 채용연계형 인턴 모집 공고(~10/25) (Distance: 21.3632)
Rank 8: [컴투스홀딩스] 산업기능요원 채용 (Distance: 21.3632)
Rank 9: LG유플러스 21년 하반기 CTO 신입 개발자 수시채용(~21.12.15 :11:00까지) (Distance: 21.3632)
Rank 10: [넷마블네오]2021년 신입공개채용 모집(~10/11 17:00까지) (Distance: 21.3632)
